In [6]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('dataset/CSE-CIC-IDS2018'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


dataset/CSE-CIC-IDS2018\02-14-2018.csv
dataset/CSE-CIC-IDS2018\02-15-2018.csv
dataset/CSE-CIC-IDS2018\02-16-2018.csv
dataset/CSE-CIC-IDS2018\02-20-2018.csv
dataset/CSE-CIC-IDS2018\02-21-2018.csv
dataset/CSE-CIC-IDS2018\02-22-2018.csv
dataset/CSE-CIC-IDS2018\02-23-2018.csv
dataset/CSE-CIC-IDS2018\02-28-2018.csv
dataset/CSE-CIC-IDS2018\03-01-2018.csv
dataset/CSE-CIC-IDS2018\03-02-2018.csv


In [7]:
import numpy as np
import pandas as pd 

import os
import gc
import torch  


# ✅ Check GPU Support Availability
print(f"Is support GPU: {torch.cuda.is_available()}")
# ✅ Check GPU Availability
print(f"🚀 GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🖥️ GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("❌ No GPU Detected. Running on CPU.")

gc.collect()

Is support GPU: True
🚀 GPU Available: True
🖥️ GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU


1951

In [13]:
import gc
import os
import psutil
import IPython

# ✅ Function to Monitor RAM Usage
def check_ram():
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / 1024 ** 3  # Convert to GB
    print(f"🔍 Current RAM Usage: {ram_usage:.2f} GB")

# ✅ Function to Clean RAM
def clean_ram():
    gc.collect()  # ✅ Free Unused Memory
    IPython.display.clear_output(wait=True)  # ✅ Clear Output (Optional)
    check_ram()  # ✅ Show Updated RAM Usage

# Example Usage:
# Run this after every large operation (like merging datasets, model training)
check_ram()  # Before cleaning
clean_ram()  # After cleaning

🔍 Current RAM Usage: 1.97 GB


In [ ]:
import pandas as pd
import os
import gc

chunk_list = [] 
chunksize = 100000 
# Định nghĩa thư mục chứa dữ liệu CSV
data_directory = 'dataset/CSE-CIC-IDS2018' 
# Định nghĩa tên file cần bỏ qua
file_to_skip = '02-20-2018.csv'

print(f"Bắt đầu tải dữ liệu từ: {data_directory} (theo chunk)")
print(f"Kích thước chunk: {chunksize}")
print(f"Sẽ bỏ qua file: {file_to_skip}")

# Duyệt qua cây thư mục
for dirname, _, filenames in os.walk(data_directory):
    for filename in filenames:
        # Bỏ qua file chỉ định
        if filename == file_to_skip:
            full_path_skipped = os.path.join(dirname, filename)
            print(f"Bỏ qua: {full_path_skipped}")
            continue 

        # Chỉ xử lý các file CSV
        if filename.lower().endswith('.csv'):
            file_path = os.path.join(dirname, filename)
            print(f"Đang xử lý file: {file_path}")
            
            try:
                # pd.read_csv với chunksize trả về một iterator
                chunk_iterator = pd.read_csv(
                    file_path, 
                    chunksize=chunksize, 
                    low_memory=False 
                )
                
                file_chunk_count = 0 # Đếm số chunk đã đọc từ file hiện tại
                # Lặp qua từng chunk trong file
                for chunk in chunk_iterator:
                    # Kiểm tra chunk rỗng
                    if chunk.empty:
                        print(f"-> Chunk rỗng trong file {filename}. Bỏ qua.")
                        continue
                    chunk_list.append(chunk)
                    file_chunk_count += 1
                    del chunk
                    gc.collect()
                    
                print(f"Đã xử lý {file_chunk_count} chunk(s) từ file {filename}.")

            except pd.errors.EmptyDataError:
                print(f"Cảnh báo: File {filename} trống hoặc không có dữ liệu. Bỏ qua.")
            # Bắt các lỗi liên quan đến bộ nhớ cụ thể hơn nếu muốn
            except MemoryError:
                 print(f"Lỗi MemoryError khi đọc file {filename}. Có thể chunksize vẫn quá lớn hoặc RAM quá thấp.")
            except Exception as e:
                print(f"Lỗi khi đọc hoặc xử lý file {filename}: {e}")
        else:
            print(f"-> Bỏ qua file không phải CSV: {filename}")

print("\nHoàn thành việc đọc tất cả các file theo chunk.")

# Bước này sẽ diễn ra MỘT LẦN DUY NHẤT sau khi đọc xong tất cả, hiệu quả hơn về bộ nhớ
print("🚀 Bắt đầu gộp các chunk thành DataFrame cuối cùng...")
df_main = pd.DataFrame(data=None) 

if chunk_list: 
    try:
        df_main = pd.concat(chunk_list, ignore_index=True)
        print("✅ Gộp chunk thành công!")
        del chunk_list
        gc.collect()
    except MemoryError:
        print(f"Lỗi MemoryError khi gộp các chunk! Dữ liệu tổng hợp vẫn quá lớn cho RAM.")
        print(f"ố lượng chunk đã đọc: {len(chunk_list)}")
    except Exception as e:
         print(f"❌ Lỗi khi gộp các chunk: {e}")
else:
    print("Không có chunk nào được đọc thành công. DataFrame 'df_main' sẽ rỗng.")

if df_main.empty:
    print("CẢNH BÁO: DataFrame 'df_main' cuối cùng vẫn rỗng hoặc không thể gộp.")
    print("Vui lòng kiểm tra lại đường dẫn, nội dung file, lỗi bộ nhớ và thử điều chỉnh chunksize hoặc dtypes.")
else:
    print(f"Kích thước DataFrame cuối cùng: {df_main.shape}")
    print("DataFrame 'df_main' đã sẵn sàng.")

🚀 Bắt đầu tải dữ liệu từ: dataset/CSE-CIC-IDS2018 (theo chunk)
   Kích thước chunk: 100000
   Sẽ bỏ qua file: 02-20-2018.csv
   📂 Đang xử lý file: dataset/CSE-CIC-IDS2018\02-14-2018.csv
      ✅ Đã xử lý 11 chunk(s) từ file 02-14-2018.csv.
   📂 Đang xử lý file: dataset/CSE-CIC-IDS2018\02-15-2018.csv
      ✅ Đã xử lý 11 chunk(s) từ file 02-15-2018.csv.
   📂 Đang xử lý file: dataset/CSE-CIC-IDS2018\02-16-2018.csv
      ✅ Đã xử lý 11 chunk(s) từ file 02-16-2018.csv.
   ➡️ Bỏ qua: dataset/CSE-CIC-IDS2018\02-20-2018.csv
   📂 Đang xử lý file: dataset/CSE-CIC-IDS2018\02-21-2018.csv
      ✅ Đã xử lý 11 chunk(s) từ file 02-21-2018.csv.
   📂 Đang xử lý file: dataset/CSE-CIC-IDS2018\02-22-2018.csv
      ✅ Đã xử lý 11 chunk(s) từ file 02-22-2018.csv.
   📂 Đang xử lý file: dataset/CSE-CIC-IDS2018\02-23-2018.csv
      ✅ Đã xử lý 11 chunk(s) từ file 02-23-2018.csv.
   📂 Đang xử lý file: dataset/CSE-CIC-IDS2018\02-28-2018.csv
      ✅ Đã xử lý 7 chunk(s) từ file 02-28-2018.csv.
   📂 Đang xử lý file: dat

In [ ]:
import numpy as np
import pandas as pd

print("\n🚀 **Final Data Verification Started...** 🚀")

### ✅ 1. CHECK FOR DUPLICATE ROWS
print("\n🔍 Checking for Duplicate Rows...")
duplicates = df_main.duplicated().sum()
print(f"✅ Total Duplicate Rows: {duplicates}")

### ✅ 2. CHECK FOR REPEATED HEADERS
print("\n🔍 Checking for Repeated Headers...")

# --- THÊM KIỂM TRA NÀY ---
if df_main.empty:
    print("⚠️ Warning: DataFrame is empty. Cannot check for repeated headers in row 1.")
    # Bạn có thể gán header_duplicates = 0 hoặc xử lý phù hợp nếu cần
    header_duplicates = 0
else:
    try:
        # Chuyển hàng đầu tiên thành chuỗi để so sánh an toàn hơn với tên cột (là chuỗi)
        first_row_as_str = df_main.iloc[0].astype(str)
        header_duplicates = (first_row_as_str == df_main.columns).sum()

        # Điều kiện kiểm tra (có thể giữ nguyên hoặc điều chỉnh)
        if header_duplicates >= len(df_main.columns) // 2:
            print(f"⚠️ Warning: Dataset might have repeated headers in row 1!") # Có thể có header lặp lại ở hàng 1
        else:
            print("✅ No apparent repeated headers found in row 1.") # Không thấy header lặp lại rõ ràng ở hàng 1
    except Exception as e:
        print(f"⚠️ Error checking repeated headers in row 1: {e}")
        header_duplicates = 0 # Xử lý lỗi

### ✅ 3. CHECK FOR MISSING / NULL VALUES
print("\n🔍 Checking for Missing Values (NaN)...")
missing_values = df_main.isnull().sum()
missing_values = missing_values[missing_values > 0]
if not missing_values.empty:
    print(missing_values)
else:
    print("✅ No Missing Values Found!")

### ✅ 4. CHECK FOR INFINITE VALUES (INF, -INF)
print("\n🔍 Checking for Infinite Values...")
inf_values = (df_main == np.inf).sum() + (df_main == -np.inf).sum()
inf_values = inf_values[inf_values > 0]
if not inf_values.empty:
    print(inf_values)
else:
    print("✅ No Infinite Values Found!")

### ✅ 5. CHECK FOR EMPTY ROWS
print("\n🔍 Checking for Completely Empty Rows...")
empty_rows = (df_main.isnull().all(axis=1)).sum()
print(f"✅ Total Empty Rows: {empty_rows}")


### ✅ 7. CHECK FOR EXTRA SPACES IN ATTACK LABELS
print("\n🔍 Checking for Extra Spaces in Attack Labels...")

try:
    # Chuyển đổi kiểu dữ liệu của cột sang string trước khi strip
    df_main["Label"] = df_main["Label"].astype(str).str.strip()
    print("Unique labels after stripping:")
    print(df_main["Label"].unique())
except Exception as e:
    print(f"⚠️ Error processing 'Label' column: {e}")

### ✅ 8. CHECK FOR NON-NUMERIC VALUES IN NUMERIC COLUMNS
print("\n🔍 Checking for Non-Numeric Values in Numeric Columns...")
for col in df_main.columns:
    if col not in ["Label", "Timestamp"]:  # Exclude categorical columns
        df_main[col] = pd.to_numeric(df_main[col], errors="coerce")

non_numeric_counts = df_main.isnull().sum()
non_numeric_counts = non_numeric_counts[non_numeric_counts > 0]
if not non_numeric_counts.empty:
    print(non_numeric_counts)
else:
    print("✅ No Non-Numeric Values Found!")

print("\n🎉 **Final Verification Complete!** 🎉")


🚀 **Final Data Verification Started...** 🚀

🔍 Checking for Duplicate Rows...
✅ Total Duplicate Rows: 410731

🔍 Checking for Repeated Headers...
✅ No apparent repeated headers found in row 1.

🔍 Checking for Missing Values (NaN)...
Flow Byts/s    22954
dtype: int64

🔍 Checking for Infinite Values...
Flow Byts/s    10772
Flow Pkts/s    28975
dtype: int64

🔍 Checking for Completely Empty Rows...
✅ Total Empty Rows: 0

🔍 Checking for Extra Spaces in Attack Labels...
Unique labels after stripping:
['Benign' 'FTP-BruteForce' 'SSH-Bruteforce' 'DoS attacks-GoldenEye'
 'DoS attacks-Slowloris' 'DoS attacks-SlowHTTPTest' 'DoS attacks-Hulk'
 'Label' 'DDOS attack-LOIC-UDP' 'DDOS attack-HOIC' 'Brute Force -Web'
 'Brute Force -XSS' 'SQL Injection' 'Infilteration' 'Bot']

🔍 Checking for Non-Numeric Values in Numeric Columns...
Dst Port         59
Protocol         59
Flow Duration    59
Tot Fwd Pkts     59
Tot Bwd Pkts     59
                 ..
Active Min       59
Idle Mean        59
Idle Std        

In [3]:
import gc
import os
import psutil
import IPython

# ✅ Function to Monitor RAM Usage
def check_ram():
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / 1024 ** 3  # Convert to GB
    print(f"🔍 Current RAM Usage: {ram_usage:.2f} GB")

# ✅ Function to Clean RAM
def clean_ram():
    gc.collect()  # ✅ Free Unused Memory
    IPython.display.clear_output(wait=True)  # ✅ Clear Output (Optional)
    check_ram()  # ✅ Show Updated RAM Usage

# Example Usage:
# Run this after every large operation (like merging datasets, model training)
check_ram()  # Before cleaning
clean_ram()  # After cleaning

🔍 Current RAM Usage: 4.29 GB


In [ ]:
import pandas as pd
import os
import gc

data_dir = "dataset/CSE-CIC-IDS2018"
output_path = "dataset/working2/merged_clean(real).csv"

# Kích thước mỗi chunk (số dòng đọc mỗi lần, tùy chỉnh theo RAM của bạn)
chunksize = 80000 

# Biến cờ để kiểm tra xem đã ghi header chưa
header_written = False
# Danh sách các cột thực tế sẽ được lấy từ file đầu tiên
actual_columns = None

print("Starting Optimized Processing (Without Predefined dtypes)...")

# Lấy danh sách file .csv
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

for filename in csv_files:
    if filename == "02-20-2018.csv":
        print(f"Skipping {filename} (Too Large & Mostly Benign)")
        continue
    file_path = os.path.join(data_dir, filename)
    print(f"Processing Chunks for: {file_path}")
    try:
        chunk_iterator = pd.read_csv(
            file_path,
            chunksize=chunksize,
            low_memory=False
        )
        for i, chunk in enumerate(chunk_iterator):
            if not header_written:
                actual_columns = chunk.columns 
                chunk.head(0).to_csv(output_path, mode='w', header=True, index=False)
                header_written = True
                print(f"Header written to {output_path}")
            if actual_columns is not None:
                try:
                    chunk = chunk[actual_columns]
                except KeyError as e_col:
                    missing_cols = set(actual_columns) - set(chunk.columns)
                    print(f"Warning: Chunk {i} in {filename} missing columns: {missing_cols}. Skipping chunk or filling NaNs might be needed.")
                    for col in missing_cols:
                         chunk[col] = pd.NA
                    chunk = chunk[actual_columns]
            try:
                if 'Dst Port' in chunk.columns:
                    chunk = chunk[chunk['Dst Port'].astype(str) != 'Dst Port']
                else:
                     print(f"Warning: 'Dst Port' column not found in chunk {i} of {filename}. Cannot remove repeated headers based on it.")
            except Exception as e_clean:
                 print(f"Error cleaning header in chunk {i} of {filename}: {e_clean}")
            chunk.to_csv(output_path, mode='a', header=False, index=False)
            del chunk
            gc.collect()
        print(f"Finished processing for: {filename}")
    except FileNotFoundError:
        print(f"Error: File not found - {file_path}")
    except pd.errors.EmptyDataError:
        print(f"Warning: File is empty - {file_path}")
    except Exception as e:
        print(f"An error occurred processing {filename}: {e}")
    finally:
        gc.collect()


print("\nOptimized Processing Complete!")
print(f"Cleaned data appended to: {output_path}")

🚀 Starting Optimized Processing (Without Predefined dtypes)...
📂 Processing Chunks for: dataset/CSE-CIC-IDS2018\02-14-2018.csv
✅ Header written to dataset/working2/merged_clean(real).csv
✅ Finished processing chunks for: 02-14-2018.csv
📂 Processing Chunks for: dataset/CSE-CIC-IDS2018\02-15-2018.csv
✅ Finished processing chunks for: 02-15-2018.csv
📂 Processing Chunks for: dataset/CSE-CIC-IDS2018\02-16-2018.csv
✅ Finished processing chunks for: 02-16-2018.csv
❌ Skipping 02-20-2018.csv (Too Large & Mostly Benign)
📂 Processing Chunks for: dataset/CSE-CIC-IDS2018\02-21-2018.csv
✅ Finished processing chunks for: 02-21-2018.csv
📂 Processing Chunks for: dataset/CSE-CIC-IDS2018\02-22-2018.csv
✅ Finished processing chunks for: 02-22-2018.csv
📂 Processing Chunks for: dataset/CSE-CIC-IDS2018\02-23-2018.csv
✅ Finished processing chunks for: 02-23-2018.csv
📂 Processing Chunks for: dataset/CSE-CIC-IDS2018\02-28-2018.csv
✅ Finished processing chunks for: 02-28-2018.csv
📂 Processing Chunks for: dataset

In [4]:
import gc
import os
import psutil
import IPython

def check_ram():
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / 1024 ** 3 
    print(f"🔍 Current RAM Usage: {ram_usage:.2f} GB")

def clean_ram():
    gc.collect()
    IPython.display.clear_output(wait=True)  # ✅ Clear Output (Optional)
    check_ram()

check_ram() 
clean_ram() 

🔍 Current RAM Usage: 4.63 GB


In [ ]:
import pandas as pd
import numpy as np
import gc
import time # Thêm thư viện time để đo thời gian

# --- Hàm xử lý cho mỗi chunk ---
def process_chunk(chunk_df):
    """Áp dụng các bước làm sạch cơ bản cho một DataFrame chunk."""
    try:
        # Bước 1: Xóa tiêu đề lặp lại
        chunk_df.drop(chunk_df[chunk_df["Label"] == "Label"].index, inplace=True, errors='ignore')
        # Bước 2: Xử lý giá trị vô cực & NaN
        chunk_df.replace([np.inf, -np.inf], np.nan, inplace=True)
        # Bước 3: Xóa hàng chứa toàn NaN
        chunk_df.dropna(inplace=True)
        # Bước 4: Xóa hàng trùng lặp 
        chunk_df.drop_duplicates(inplace=True)
        # Bước 5: Chuyển đổi cột số học và tối ưu kiểu dữ liệu (downcast)
        potential_non_numeric = ["Label", "Protocol", "Timestamp"]
        numeric_cols = chunk_df.columns.difference(potential_non_numeric, sort=False)
        # Chỉ chuyển đổi các cột thực sự tồn tại trong chunk
        cols_to_convert = [col for col in numeric_cols if col in chunk_df.columns]
        if cols_to_convert:
            chunk_df[cols_to_convert] = chunk_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
            # Tối ưu kiểu dữ liệu (downcasting)
            for col in cols_to_convert:
                 if col in chunk_df.columns:
                    if chunk_df[col].dtype == 'float64':
                        chunk_df[col] = pd.to_numeric(chunk_df[col], downcast='float')
                    elif chunk_df[col].dtype == 'int64':
                        chunk_df[col] = pd.to_numeric(chunk_df[col], downcast='integer')
        # Bước 6: Xóa cột "Timestamp" nếu tồn tại
        if "Timestamp" in chunk_df.columns:
            chunk_df.drop(columns=["Timestamp"], inplace=True)
        # Tạm thời giữ lại cột Protocol dưới dạng category để xử lý sau khi kết hợp
        if "Protocol" in chunk_df.columns:
            chunk_df["Protocol"] = chunk_df["Protocol"].astype('category')
    except Exception as e:
        print(f"⚠️ Lỗi khi xử lý chunk: {e}. Bỏ qua chunk này hoặc xử lý lỗi...")
        # Trả về DataFrame rỗng hoặc xử lý lỗi phù hợp
        return pd.DataFrame() # Trả về DF rỗng để tránh lỗi khi concat
    return chunk_df

# --- Chương trình chính ---
start_time = time.time()
print("🔄 Bắt đầu quá trình xử lý dữ liệu theo từng phần (chunking)...")

chunksize = 100000
processed_chunks = []
file_path = "dataset/working2/merged_clean(real).csv"
output_path = "dataset/working2/cleaned_optimized_dataset_chunked.csv"
try:
    chunk_iterator = pd.read_csv(file_path, chunksize=chunksize, low_memory=False)
    total_chunks = 0

    print(f"Đang đọc tệp '{file_path}' với kích thước chunk là {chunksize}...")

    for i, chunk in enumerate(chunk_iterator):
        total_chunks = i + 1
        print(f"Đang xử lý chunk {total_chunks}...")
        start_chunk_time = time.time()
        processed_chunk = process_chunk(chunk.copy()) 
        if not processed_chunk.empty:
             processed_chunks.append(processed_chunk)

        chunk_time = time.time() - start_chunk_time
        print(f"-> Chunk {total_chunks} xử lý xong trong {chunk_time:.2f} giây.")

        # Thu gom rác định kỳ để giải phóng bộ nhớ
        if total_chunks % 5 == 0:
            print(f"Đang thực hiện thu gom rác (GC) sau chunk {total_chunks}...")
            gc.collect()

    if not processed_chunks:
        print("Không có chunk nào được xử lý thành công. Kiểm tra lại dữ liệu đầu vào hoặc hàm process_chunk.")
        exit()

    print(f"\nĐã xử lý tổng cộng {total_chunks} chunk. Bắt đầu kết hợp...")
    start_concat_time = time.time()

    # Kết hợp tất cả các chunk đã xử lý
    df_main = pd.concat(processed_chunks, ignore_index=True)

    concat_time = time.time() - start_concat_time
    print(f"Kết hợp {len(processed_chunks)} chunk thành công trong {concat_time:.2f} giây.")

    # Giải phóng bộ nhớ ngay lập tức từ danh sách chunk
    del processed_chunks
    gc.collect()
    print("Đã giải phóng bộ nhớ từ danh sách chunk.")

    # --- Thực hiện các bước làm sạch cuối cùng sau khi kết hợp ---
    print("\n🧹 Thực hiện các bước làm sạch cuối cùng trên DataFrame đã kết hợp...")

    # Bước 4: Xóa các hàng trùng lặp *tổng thể*
    rows_before_dup_drop = len(df_main)
    df_main.drop_duplicates(inplace=True)
    print(f"Bước 4: Đã xóa {rows_before_dup_drop - len(df_main)} hàng trùng lặp tổng thể.")

    # Bước 5: Xóa các cột có giá trị không đổi
    numeric_cols_final = df_main.select_dtypes(include=np.number).columns
    if not numeric_cols_final.empty:
        variances = df_main[numeric_cols_final].var()
        constant_columns = variances[variances == 0].index
        if len(constant_columns) > 0:
            df_main.drop(columns=constant_columns, inplace=True)
            print(f"Bước 5: Đã xóa {len(constant_columns)} cột không đổi: {list(constant_columns)}")
        else:
            print("Bước 5: Không tìm thấy cột nào có giá trị không đổi.")
    else:
        print("Bước 5: Không có cột số học nào để kiểm tra phương sai.")


    # Bước 6: Mã hóa "Protocol" bằng One-Hot Encoding
    if "Protocol" in df_main.columns:
        if not pd.api.types.is_categorical_dtype(df_main["Protocol"]):
            df_main["Protocol"] = df_main["Protocol"].astype('category')
        print("Bước 6a: Đã chuyển/xác nhận cột 'Protocol' sang kiểu 'category'.")
        df_main = pd.get_dummies(df_main, columns=["Protocol"], prefix="Protocol", dtype=np.int8)
        print("Bước 6b: Đã mã hóa 'Protocol' bằng One-Hot Encoding (kiểu int8).")
    else:
        print("Bước 6: Không tìm thấy cột 'Protocol' để mã hóa.")

    # Bước 7: Đảm bảo cột "Label" là cột cuối cùng
    if "Label" in df_main.columns:
        label_col = df_main.pop("Label")
        df_main.insert(len(df_main.columns), "Label", label_col)
        print("Bước 7: Đã di chuyển cột 'Label' về cuối.")
    else:
        print("Bước 7: Không tìm thấy cột 'Label'.")

    # Bước 10: Tinh chỉnh kiểu dữ liệu cuối cùng cho 'Dst Port'
    if "Dst Port" in df_main.columns:
        print("Bước 10: Tinh chỉnh kiểu dữ liệu 'Dst Port'...")
        # Chuyển sang số học nếu chưa phải, ép lỗi thành NaN
        df_main["Dst Port"] = pd.to_numeric(df_main["Dst Port"], errors='coerce')
        # Xóa các hàng mà 'Dst Port' trở thành NaN sau khi ép kiểu
        rows_before_na = len(df_main)
        df_main.dropna(subset=["Dst Port"], inplace=True)
        if len(df_main) < rows_before_na:
             print(f"-> Đã xóa {rows_before_na - len(df_main)} hàng có 'Dst Port' không hợp lệ.")

        # Kiểm tra min/max để chọn kiểu integer phù hợp nhất
        min_port = df_main["Dst Port"].min()
        max_port = df_main["Dst Port"].max()

        if pd.notna(min_port) and pd.notna(max_port):
            if min_port >= 0 and max_port <= 65535:
                df_main["Dst Port"] = df_main["Dst Port"].astype(np.uint16)
                print(f"-> Đã chuyển đổi 'Dst Port' sang uint16.")
            elif min_port >= 0 and max_port <= 4294967295: # Giới hạn của uint32
                 df_main["Dst Port"] = df_main["Dst Port"].astype(np.uint32)
                 print(f"-> Đã chuyển đổi 'Dst Port' sang uint32.")
            else:
                 # Sử dụng downcast để Pandas tự chọn kiểu int phù hợp nhất
                 df_main["Dst Port"] = pd.to_numeric(df_main["Dst Port"], downcast='integer')
                 print(f"-> Đã chuyển đổi 'Dst Port' sang kiểu integer tối ưu: {df_main['Dst Port'].dtype}.")
        else:
             print("-> Không thể xác định min/max cho 'Dst Port' (có thể do cột rỗng). Giữ nguyên kiểu hiện tại.")
    else:
        print("Bước 10: Không tìm thấy cột 'Dst Port'.")

    # Tối ưu hóa lại tất cả các cột số học một lần nữa sau các thay đổi
    print("   - Tối ưu hóa lại kiểu dữ liệu số học cuối cùng...")
    for col in df_main.select_dtypes(include=['float64', 'int64']).columns:
        if col in df_main.columns: # Kiểm tra cột còn tồn tại không
            if df_main[col].dtype == 'float64':
                df_main[col] = pd.to_numeric(df_main[col], downcast='float')
            elif df_main[col].dtype == 'int64':
                df_main[col] = pd.to_numeric(df_main[col], downcast='integer')

    print("\n📊 Thông tin DataFrame cuối cùng:")
    # Bước 11: Hiển thị thông tin cuối cùng của DataFrame
    df_main.info(memory_usage='deep')

    # Bước 12: Lưu dữ liệu đã làm sạch
    print(f"\n💾 Đang lưu DataFrame đã xử lý vào '{output_path}'...")
    start_save_time = time.time()
    df_main.to_csv(output_path, index=False)
    save_time = time.time() - start_save_time
    print(f"✅ Dữ liệu đã xử lý theo chunk được lưu thành công trong {save_time:.2f} giây.")

except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy tệp '{file_path}'. Hãy đảm bảo đường dẫn chính xác.")
except pd.errors.EmptyDataError:
     print(f"❌ Lỗi: Tệp '{file_path}' trống hoặc không chứa dữ liệu.")
except Exception as e:
    print(f"❌ Đã xảy ra lỗi không mong muốn trong quá trình xử lý: {e}")
    import traceback
    traceback.print_exc() # In chi tiết lỗi để gỡ rối

finally:
    if 'df_main' in locals():
        del df_main
    if 'processed_chunks' in locals() and processed_chunks is not None:
         del processed_chunks
    gc.collect()
    end_time = time.time()
    total_time = end_time - start_time
    print(f"\n✨ Quá trình hoàn tất trong {total_time:.2f} giây. Bộ nhớ đã được giải phóng.")

🔄 Bắt đầu quá trình xử lý dữ liệu theo từng phần (chunking)...
   - Đang đọc tệp 'dataset/working2/merged_clean(real).csv' với kích thước chunk là 100000...
   - Đang xử lý chunk 1...
     -> Chunk 1 xử lý xong trong 0.46 giây.
   - Đang xử lý chunk 2...
     -> Chunk 2 xử lý xong trong 0.51 giây.
   - Đang xử lý chunk 3...
     -> Chunk 3 xử lý xong trong 0.54 giây.
   - Đang xử lý chunk 4...
     -> Chunk 4 xử lý xong trong 0.63 giây.
   - Đang xử lý chunk 5...
     -> Chunk 5 xử lý xong trong 0.81 giây.
   - Đang thực hiện thu gom rác (GC) sau chunk 5...
   - Đang xử lý chunk 6...
     -> Chunk 6 xử lý xong trong 0.88 giây.
   - Đang xử lý chunk 7...
     -> Chunk 7 xử lý xong trong 0.86 giây.
   - Đang xử lý chunk 8...
     -> Chunk 8 xử lý xong trong 0.96 giây.
   - Đang xử lý chunk 9...
     -> Chunk 9 xử lý xong trong 0.93 giây.
   - Đang xử lý chunk 10...
     -> Chunk 10 xử lý xong trong 0.93 giây.
   - Đang thực hiện thu gom rác (GC) sau chunk 10...
   - Đang xử lý chunk 11..

C:\Users\linhv\AppData\Local\Temp\ipykernel_20052\3323830457.py:141: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df_main["Protocol"]):


   - Bước 8b: Đã mã hóa 'Protocol' bằng One-Hot Encoding (kiểu int8).
   - Bước 9: Đã di chuyển cột 'Label' về cuối.
   - Bước 10: Tinh chỉnh kiểu dữ liệu 'Dst Port'...
     -> Đã chuyển đổi 'Dst Port' sang uint16.
   - Tối ưu hóa lại kiểu dữ liệu số học cuối cùng...

📊 Thông tin DataFrame cuối cùng:
<class 'pandas.core.frame.DataFrame'>
Index: 5881823 entries, 0 to 7864305
Data columns (total 73 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Dst Port           uint16 
 1   Flow Duration      int64  
 2   Tot Fwd Pkts       int32  
 3   Tot Bwd Pkts       int32  
 4   TotLen Fwd Pkts    int32  
 5   TotLen Bwd Pkts    float64
 6   Fwd Pkt Len Max    int32  
 7   Fwd Pkt Len Min    int16  
 8   Fwd Pkt Len Mean   float64
 9   Fwd Pkt Len Std    float64
 10  Bwd Pkt Len Max    int32  
 11  Bwd Pkt Len Min    int16  
 12  Bwd Pkt Len Mean   float64
 13  Bwd Pkt Len Std    float32
 14  Flow Byts/s        float64
 15  Flow Pkts/s        float64
 16  Flow IAT 

In [11]:
import pandas as pd

# Load the cleaned dataset
file_path = "dataset/working2/cleaned_dataset.csv"  # Change this path if needed
df = pd.read_csv(file_path)

# Get the feature names (excluding the label column)
feature_names = df.columns.tolist()
feature_names.remove("Label")

print("✅ Features Used for Training:")
print(feature_names)

# Save feature names to a text file (to transfer to Ubuntu VM)
with open("dataset/working2/feature_list.txt", "w") as f:
    for feature in feature_names:
        f.write(feature + "\n")

print("\n✅ Feature names saved to feature_list.txt")

✅ Features Used for Training:
['Dst Port', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Init Fwd Wi

NORMALIZE DATA AND SPLIT DATASET

In [ ]:
import pandas as pd
import gc
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import numpy as np
import joblib

# ✅ Load Cleaned Data
cleaned_file = "dataset/working2/cleaned_dataset.csv"
print(f"📂 Loading Dataset: {cleaned_file} ...")
df = pd.read_csv(cleaned_file)
print(f"✅ Dataset Loaded! Shape: {df.shape}")

# ✅ (1) Encode Label Column (Multi-Class)
label_mapping = {
    "Benign": 0,
    "Bot": 1,
    "DDOS attack-HOIC": 2,
    "DDOS attack-LOIC-UDP": 3,
    "DoS attacks-GoldenEye": 4,
    "DoS attacks-Hulk": 5,
    "DoS attacks-SlowHTTPTest": 6,
    "DoS attacks-Slowloris": 7,
    "FTP-BruteForce": 8,
    "Infilteration": 9,
    "SSH-Bruteforce": 10,
    "Brute Force -Web": 11,
    "Brute Force -XSS": 12,
    "SQL Injection": 13
}
df["Label"] = df["Label"].map(label_mapping)

# ✅ (2) Train-Test Split FIRST (BEFORE Normalization)
df_train, df_test = train_test_split(df, test_size=0.2, random_state=20, stratify=df["Label"])
print(f"✅ Train Shape: {df_train.shape}, Test Shape: {df_test.shape}")

# Chuẩn hóa các cột số
numeric_cols = df_train.columns.difference(["Label"])  # Exclude Label column
scaler = MinMaxScaler()

# Fit mô hình scaler trên tập dữ liệu huấn luyện
scaler.fit(df_train[numeric_cols])

# Biến đổi tập dữ liệu train, test cùng 1 mô hình scaler
df_train[numeric_cols] = scaler.transform(df_train[numeric_cols])
df_test[numeric_cols] = scaler.transform(df_test[numeric_cols])

print(f"Normalization Complete!")
print(f"Train data range: {df_train[numeric_cols].min().min():.2f} to {df_train[numeric_cols].max().max():.2f}")
print(f"Test data range: {df_test[numeric_cols].min().min():.2f} to {df_test[numeric_cols].max().max():.2f}")

# Lưu mô hình scaler
SCALER_PATH = "dataset/working2/scaler.pkl"
joblib.dump(scaler, SCALER_PATH)
print(f"✅ Scaler saved at: {SCALER_PATH}")

# ✅ (5) Handle Imbalance Using SMOTE (Now applied to NORMALIZED data)
X_train = df_train.drop(columns=["Label"]).values
y_train = df_train["Label"].values

# Define rare attack types
rare_classes = [11, 12, 13]  # Brute Force -Web, Brute Force -XSS, SQL Injection

print("\n🚀 Applying SMOTE ONLY on Rare Attack Types (to normalized data)...")
smote = SMOTE(sampling_strategy={cls: 5000 for cls in rare_classes}, 
              random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Convert back to DataFrame
df_train_balanced = pd.DataFrame(X_resampled, columns=df_train.columns[:-1])
df_train_balanced["Label"] = y_resampled
print(f"✅ SMOTE Applied! New Train Shape: {df_train_balanced.shape}")

# Verify SMOTE didn't break normalization
print(f"Post-SMOTE range check: {df_train_balanced.iloc[:, :-1].min().min():.2f} to {df_train_balanced.iloc[:, :-1].max().max():.2f}")

# ✅ (6) Save Processed Data
df_train_balanced.to_csv("dataset/working2/train_balanced1.csv", index=False)
df_test.to_csv("dataset/working2/test1.csv", index=False)
print("\n✅ Train & Test Data Saved Successfully!")

# ✅ (7) Verification
print("\n🔍 Label Distribution in Train Set (After SMOTE):")
print(df_train_balanced["Label"].value_counts())

print("\n🔍 Label Distribution in Test Set (Original Distribution):")
print(df_test["Label"].value_counts())

# Cleanup
del df, df_train, X_train, y_train, X_resampled, y_resampled
gc.collect()

📂 Loading Dataset: dataset/working2/cleaned_dataset.csv ...
✅ Dataset Loaded! Shape: (5881823, 73)
✅ Train Shape: (4705458, 73), Test Shape: (1176365, 73)
✅ Normalization Complete!
Train data range: 0.00 to 1.00
Test data range: 0.00 to 3.46
✅ Scaler saved at: dataset/working2/scaler.pkl

🚀 Applying SMOTE ONLY on Rare Attack Types (to normalized data)...
✅ SMOTE Applied! New Train Shape: (4719765, 73)
Post-SMOTE range check: 0.00 to 1.00

✅ Train & Test Data Saved Successfully!

🔍 Label Distribution in Train Set (After SMOTE):
Label
0     4069824
2      173921
5      116159
1      115628
9      111473
10      75238
4       33125
7        7926
11       5000
12       5000
13       5000
3        1384
6          44
8          43
Name: count, dtype: int64

🔍 Label Distribution in Test Set (Original Distribution):
Label
0     1017456
2       43480
5       29040
1       28907
9       27868
10      18810
4        8281
7        1982
3         346
11        111
12         46
13         17
6     

0

In [13]:
import torch
label_mapping = {
    "Benign": 0,
    "Bot": 1,
    "DDOS attack-HOIC": 2,
    "DDOS attack-LOIC-UDP": 3,
    "DoS attacks-GoldenEye": 4,
    "DoS attacks-Hulk": 5,
    "DoS attacks-SlowHTTPTest": 6,
    "DoS attacks-Slowloris": 7,
    "FTP-BruteForce": 8,
    "Infilteration": 9,
    "SSH-Bruteforce": 10,
    "Brute Force -Web": 11,
    "Brute Force -XSS": 12,
    "SQL Injection": 13
}
torch.save(label_mapping, "dataset/working2/label_mapping.pth")

# MODEL TRAINING

In [14]:
import gc
import torch

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Clear Python garbage collection
gc.collect()

print("✅ RAM and GPU memory cleared!")

✅ RAM and GPU memory cleared!


In [15]:
import gc
import os
import psutil
import IPython

# ✅ Function to Monitor RAM Usage
def check_ram():
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / 1024 ** 3  # Convert to GB
    print(f"🔍 Current RAM Usage: {ram_usage:.2f} GB")

# ✅ Function to Clean RAM
def clean_ram():
    gc.collect()  # ✅ Free Unused Memory
    IPython.display.clear_output(wait=True)  # ✅ Clear Output (Optional)
    check_ram()  # ✅ Show Updated RAM Usage

# Example Usage:
# Run this after every large operation (like merging datasets, model training)
check_ram()  # Before cleaning
clean_ram()  # After cleaning

🔍 Current RAM Usage: 0.20 GB


In [ ]:
import pandas as pd
import numpy as np
import gc
import time
import joblib 

# Các thuật toán và công cụ đánh giá từ Scikit-learn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
# Thư viện XGBoost
import xgboost as xgb

# Vẽ biểu đồ
import matplotlib.pyplot as plt
import seaborn as sns

# Thư viện PyTorch (chỉ dùng để tải label_mapping nếu bạn đã lưu bằng torch.save)
import torch

# --- Hàm trợ giúp để đánh giá mô hình ---
def evaluate_model(model, X_test, y_test, class_names, model_name, results_dir="dataset/working2"):
    """
    Đánh giá mô hình, in các chỉ số, lưu kết quả và vẽ ma trận nhầm lẫn.
    (Giữ nguyên nội dung hàm này)
    """
    print(f"\n🔍 Bắt đầu đánh giá mô hình: {model_name}...")
    start_eval_time = time.time()

    # Dự đoán trên tập kiểm thử
    y_pred = model.predict(X_test)

    eval_time = time.time() - start_eval_time
    print(f"   - Thời gian dự đoán: {eval_time:.2f} giây")

    # Tính toán các chỉ số
    accuracy = accuracy_score(y_test, y_pred) * 100
    try:
        # Sử dụng zero_division=0 để tránh lỗi khi một lớp không có mẫu nào được dự đoán đúng hoặc sai
        report = classification_report(y_test, y_pred, target_names=class_names, zero_division=0)
    except ValueError as e:
        print(f"⚠️ Cảnh báo khi tạo classification report cho {model_name}: {e}")
        # Cố gắng tạo báo cáo không có tên lớp nếu có lỗi
        try:
            report = classification_report(y_test, y_pred, zero_division=0)
        except Exception as report_err:
            print(f"❌ Không thể tạo classification report: {report_err}")
            report = "Không thể tạo báo cáo."

    print(f"\n🏆 Độ chính xác cuối cùng của {model_name}: {accuracy:.2f}%")
    print(f"\n📊 Báo cáo phân loại (Classification Report) của {model_name}:")
    print(report)

    # Lưu kết quả vào tệp tin
    results_filename = f"{results_dir}/{model_name.lower().replace(' ', '_')}_results.txt"
    try:
        with open(results_filename, 'w', encoding='utf-8') as f: # Thêm encoding='utf-8'
            f.write(f"{model_name} - Final Accuracy: {accuracy:.2f}%\n\n")
            f.write("Classification Report:\n")
            f.write(report)
        print(f"✅ Kết quả đã được lưu vào tệp: '{results_filename}'")
    except Exception as e:
        print(f"❌ Lỗi khi lưu kết quả cho {model_name}: {e}")

    # Vẽ và lưu ma trận nhầm lẫn (Confusion Matrix)
    cm_filename = f"{results_dir}/{model_name.lower().replace(' ', '_')}_confusion_matrix.png"
    try:
        plt.figure(figsize=(15, 12)) # Tăng kích thước để dễ đọc hơn
        conf_matrix = confusion_matrix(y_test, y_pred)
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 8}) # Giảm cỡ chữ annot
        plt.title(f'Ma trận nhầm lẫn (Confusion Matrix) - {model_name}', fontsize=14)
        plt.ylabel('Nhãn Thật (True Label)', fontsize=12)
        plt.xlabel('Nhãn Dự đoán (Predicted Label)', fontsize=12)
        plt.xticks(rotation=45, ha='right', fontsize=10) # Xoay và căn chỉnh nhãn trục X
        plt.yticks(rotation=0, fontsize=10)            # Căn chỉnh nhãn trục Y
        plt.tight_layout() # Tự động điều chỉnh bố cục
        plt.savefig(cm_filename, dpi=150) # Tăng dpi cho hình ảnh chất lượng hơn
        print(f"📈 Ma trận nhầm lẫn đã được lưu vào tệp: '{cm_filename}'")
        plt.close() # Đóng biểu đồ để giải phóng bộ nhớ
    except Exception as e:
        print(f"❌ Lỗi khi tạo/lưu ma trận nhầm lẫn cho {model_name}: {e}")


# --- Chương trình chính ---
start_total_time = time.time()

# Giải phóng bộ nhớ trước khi bắt đầu
gc.collect()

# --- Thiết lập đường dẫn ---
data_dir = "dataset/working2"
results_dir = data_dir
train_file = f"{data_dir}/train_balanced1.csv"
test_file = f"{data_dir}/test1.csv"
label_map_file = f"{data_dir}/label_mapping.pth"

# --- Tải dữ liệu ---
print("\n📂 Đang tải dữ liệu đã tiền xử lý...")
try:
    train_data = pd.read_csv(train_file)
    test_data = pd.read_csv(test_file)
    print(f"   - Kích thước tập huấn luyện: {train_data.shape}")
    print(f"   - Kích thước tập kiểm thử: {test_data.shape}")
except FileNotFoundError as e:
    print(f"❌ Lỗi: Không tìm thấy tệp dữ liệu. {e}")
    exit()
except Exception as e:
    print(f"❌ Lỗi không xác định khi tải dữ liệu: {e}")
    exit()


# --- Tải bản đồ nhãn (Label Mapping) ---
print("\n🏷️ Đang tải bản đồ nhãn...")
try:
    label_mapping = torch.load(label_map_file)
    class_names = [k for k, v in sorted(label_mapping.items(), key=lambda item: item[1])]
    num_classes = len(class_names)
    print(f"   - Tải thành công bản đồ nhãn cho {num_classes} lớp.")
    print(f"   - Các lớp: {class_names}")
except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy tệp bản đồ nhãn '{label_map_file}'.")
    max_label_train = train_data['Label'].max() if 'Label' in train_data else -1
    max_label_test = test_data['Label'].max() if 'Label' in test_data else -1
    max_label = int(max(max_label_train, max_label_test))
    if max_label >= 0:
        class_names = [f"Lop_{i}" for i in range(max_label + 1)]
        num_classes = len(class_names)
        print(f"⚠️ Đang sử dụng tên lớp mặc định: {class_names}")
    else:
        print("❌ Không thể xác định số lớp từ dữ liệu.")
        exit()
except Exception as e:
    print(f"❌ Lỗi khi tải bản đồ nhãn: {e}")
    exit()


# --- Chuẩn bị dữ liệu cho mô hình ---
print("\n⚙️ Đang chuẩn bị dữ liệu (chuyển đổi sang NumPy)...")
try:
    X_train_np = train_data.drop(columns=["Label"]).values
    y_train_np = train_data["Label"].values
    X_test_np = test_data.drop(columns=["Label"]).values
    y_test_np = test_data["Label"].values
    print(f"   - Kích thước X_train: {X_train_np.shape}, y_train: {y_train_np.shape}")
    print(f"   - Kích thước X_test: {X_test_np.shape}, y_test: {y_test_np.shape}")
except KeyError as e:
    print(f"❌ Lỗi: Không tìm thấy cột 'Label' trong DataFrame. {e}")
    exit()
except Exception as e:
    print(f"❌ Lỗi khi chuẩn bị dữ liệu NumPy: {e}")
    exit()

# Xác minh phạm vi dữ liệu
print("\n🔥 Kiểm tra phạm vi giá trị dữ liệu (sau khi tải):")
min_train, max_train = X_train_np.min(), X_train_np.max()
min_test, max_test = X_test_np.min(), X_test_np.max()
print(f"   - Phạm vi đặc trưng Train: {min_train:.4f} đến {max_train:.4f}")
print(f"   - Phạm vi đặc trưng Test: {min_test:.4f} đến {max_test:.4f}")


# Giải phóng bộ nhớ từ DataFrames không còn dùng
del train_data, test_data
gc.collect()
print("   - Đã giải phóng bộ nhớ từ Pandas DataFrames.")


# --- 1. Huấn luyện và Đánh giá Random Forest ---
print("\n" + "="*10 + " 🌳 Bắt đầu với Random Forest " + "="*10)

# --- Khởi tạo mô hình Random Forest ---
rf_model = RandomForestClassifier(
    n_estimators=200,        
    criterion='gini',       
    max_depth=30,          
    min_samples_split=2,      
    min_samples_leaf=1,      
    max_features='sqrt',      
    bootstrap=True,        
    oob_score=False,        
    random_state=42,         
    n_jobs=-1,                
    class_weight=None         
)
print(f"   - Khởi tạo Random Forest với: n_estimators={rf_model.n_estimators}, max_depth={rf_model.max_depth}, max_features='{rf_model.max_features}'")

print("   - Đang huấn luyện Random Forest...")
start_rf_train_time = time.time()
try:
    # Huấn luyện mô hình
    rf_model.fit(X_train_np, y_train_np)
    rf_train_time = time.time() - start_rf_train_time
    print(f"✅ Huấn luyện Random Forest hoàn thành trong {rf_train_time:.2f} giây.")

    # Lưu mô hình đã huấn luyện
    rf_model_path = f"{results_dir}/random_forest_model.pkl"
    joblib.dump(rf_model, rf_model_path)
    print(f"💾 Mô hình Random Forest đã được lưu vào: '{rf_model_path}'")

    # Đánh giá mô hình
    evaluate_model(rf_model, X_test_np, y_test_np, class_names, "Random Forest", results_dir=results_dir)

except MemoryError:
     print(f"❌ Lỗi bộ nhớ khi huấn luyện/đánh giá Random Forest. Hãy thử giảm n_estimators hoặc giới hạn max_depth.")
except Exception as e:
    print(f"❌ Đã xảy ra lỗi trong quá trình xử lý Random Forest: {e}")

# Giải phóng bộ nhớ
del rf_model
gc.collect()


# --- 2. Huấn luyện và Đánh giá XGBoost ---
print("\n" + "="*10 + " 🚀 Bắt đầu với XGBoost (Thử nghiệm GPU) " + "="*10)

# Kiểm tra GPU
try:
    if torch.cuda.is_available():
        print("   - Phát hiện GPU NVIDIA và CUDA.")
        tree_method_setting = 'gpu_hist' # Sử dụng GPU
        print(f"   - Sẽ sử dụng 'tree_method': '{tree_method_setting}' cho XGBoost.")
    else:
        print("   - Không phát hiện GPU NVIDIA hoặc CUDA.")
        tree_method_setting = 'hist' # Sử dụng CPU
        print(f"   - Sẽ sử dụng 'tree_method': '{tree_method_setting}' (CPU) cho XGBoost.")
except ImportError:
    print("   - Không tìm thấy PyTorch để kiểm tra CUDA, giả định sử dụng CPU.")
    tree_method_setting = 'hist' # Sử dụng CPU mặc định
    print(f"   - Sẽ sử dụng 'tree_method': '{tree_method_setting}' (CPU) cho XGBoost.")


# Khởi tạo mô hình XGBoost
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=num_classes,
    n_estimators=200,         
    learning_rate=0.1,
    max_depth=8,               
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss',
    n_jobs=-1,
    tree_method=tree_method_setting
)

print(f"   - Đang huấn luyện XGBoost sử dụng {tree_method_setting}...")
start_xgb_train_time = time.time()
try:
    # Huấn luyện mô hình
    xgb_model.fit(X_train_np, y_train_np, verbose=False)
    xgb_train_time = time.time() - start_xgb_train_time
    print(f"✅ Huấn luyện XGBoost ({tree_method_setting}) hoàn thành trong {xgb_train_time:.2f} giây.")

    # Lưu mô hình đã huấn luyện
    xgb_model_path = f"{results_dir}/xgboost_{tree_method_setting}_model.pkl"
    joblib.dump(xgb_model, xgb_model_path)
    print(f"💾 Mô hình XGBoost ({tree_method_setting}) đã được lưu vào: '{xgb_model_path}'")

    # Đánh giá mô hình
    evaluate_model(xgb_model, X_test_np, y_test_np, class_names, f"XGBoost ({tree_method_setting})", results_dir=results_dir)

except xgb.core.XGBoostError as e:
    print(f"❌ Lỗi XGBoost: {e}")
    if "CUDA" in str(e) or "GPU" in str(e):
        print("   - Lỗi có thể liên quan đến cài đặt CUDA hoặc GPU.")
    print("   - Thử chạy lại với tree_method='hist' (CPU) nếu lỗi liên quan đến GPU.")
except MemoryError:
     print(f"❌ Lỗi bộ nhớ khi huấn luyện/đánh giá XGBoost ({tree_method_setting}).")
except Exception as e:
    print(f"❌ Đã xảy ra lỗi trong quá trình xử lý XGBoost ({tree_method_setting}): {e}")


# --- Hoàn tất ---
del X_train_np, y_train_np, X_test_np, y_test_np
del rf_model
del xgb_model
gc.collect()

end_total_time = time.time()
total_script_time = end_total_time - start_total_time
print(f"\n🏁🏁🏁 Quá trình huấn luyện và đánh giá hoàn tất trong {total_script_time:.2f} giây! 🏁🏁🏁")


📂 Đang tải dữ liệu đã tiền xử lý...
   - Kích thước tập huấn luyện: (4719765, 73)
   - Kích thước tập kiểm thử: (1176365, 73)

🏷️ Đang tải bản đồ nhãn...
   - Tải thành công bản đồ nhãn cho 14 lớp.
   - Các lớp: ['Benign', 'Bot', 'DDOS attack-HOIC', 'DDOS attack-LOIC-UDP', 'DoS attacks-GoldenEye', 'DoS attacks-Hulk', 'DoS attacks-SlowHTTPTest', 'DoS attacks-Slowloris', 'FTP-BruteForce', 'Infilteration', 'SSH-Bruteforce', 'Brute Force -Web', 'Brute Force -XSS', 'SQL Injection']

⚙️ Đang chuẩn bị dữ liệu (chuyển đổi sang NumPy)...
   - Kích thước X_train: (4719765, 72), y_train: (4719765,)
   - Kích thước X_test: (1176365, 72), y_test: (1176365,)

🔥 Kiểm tra phạm vi giá trị dữ liệu (sau khi tải):
   - Phạm vi đặc trưng Train: 0.0000 đến 1.0000
   - Phạm vi đặc trưng Test: 0.0000 đến 3.4572
   - Đã giải phóng bộ nhớ từ Pandas DataFrames.

========== 🌳 Bắt đầu với Random Forest ==========
   - Khởi tạo Random Forest với: n_estimators=200, max_depth=30, max_features='sqrt'
   - Đang huấn l

d:\Do_an_tot_nghiep\apt-detection\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [05:40:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
d:\Do_an_tot_nghiep\apt-detection\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [05:40:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Huấn luyện XGBoost (gpu_hist) hoàn thành trong 241.06 giây.
💾 Mô hình XGBoost (gpu_hist) đã được lưu vào: 'dataset/working2/xgboost_gpu_hist_model.pkl'

🔍 Bắt đầu đánh giá mô hình: XGBoost (gpu_hist)...


d:\Do_an_tot_nghiep\apt-detection\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [05:44:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
d:\Do_an_tot_nghiep\apt-detection\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [05:44:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- 

   - Thời gian dự đoán: 3.03 giây

🏆 Độ chính xác cuối cùng của XGBoost (gpu_hist): 97.65%

📊 Báo cáo phân loại (Classification Report) của XGBoost (gpu_hist):
                          precision    recall  f1-score   support

                  Benign       0.97      1.00      0.99   1017456
                     Bot       1.00      1.00      1.00     28907
        DDOS attack-HOIC       1.00      1.00      1.00     43480
    DDOS attack-LOIC-UDP       1.00      1.00      1.00       346
   DoS attacks-GoldenEye       1.00      1.00      1.00      8281
        DoS attacks-Hulk       1.00      1.00      1.00     29040
DoS attacks-SlowHTTPTest       0.20      0.18      0.19        11
   DoS attacks-Slowloris       1.00      1.00      1.00      1982
          FTP-BruteForce       0.18      0.20      0.19        10
           Infilteration       0.59      0.04      0.08     27868
          SSH-Bruteforce       1.00      1.00      1.00     18810
        Brute Force -Web       0.58      0.68  